In [1]:
import pandas as pd
import copy
from datetime import datetime
from sklearn.datasets import load_iris
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
import numpy as np
import math
# y1: sevirity distance 
# y2: predict the probabilty of car accident in a specific place

In [2]:
df = pd.read_csv('US_Accidents_Dec21_updated.csv')
df.shape

(2845342, 47)

ID 保留做index
Severity 预测值
'Distance(mi)'预测值


'Start_Time', 'End_Time' 合并成一个变量叫AccourTime，做bucketize，分成2个小时的bucket
再加一个变量叫duration描述车祸产生影响的总时长
'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng'合并成一个paired tuple叫location

'Wind_Chill(F)'
'Wind_Direction'
'Weather_Condition'

'Amenity'
'Crossing'
'Give_Way'
'No_Exit'
'Junction'
'Railway'
'Roundabout'
'Station'
'Stop'
'Traffic_Calming'
'Traffic_Signal'
'Turning_Loop'

In [3]:
df.keys()

Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [4]:
WI_data = copy.deepcopy(df[df.State == 'WI'])
data = copy.deepcopy(WI_data.ID)

# Add a new row: 'Time'
Time = []
for time in WI_data.Start_Time:
    if time != None:
        Time.append(int(time[11:13])//2*2)
    else:
        Time.append(None)
WI_data['Time'] = Time

# Add a new row: 'Location'
Start_Lat = [row for row in WI_data.loc[:, 'Start_Lat']]
Start_Lng = [row for row in WI_data.loc[:, 'Start_Lng']]
Location = []
for i in range(len(Start_Lng)):
    if Start_Lat[i] == None or Start_Lng[i] == None:
        Location.append(None)
    else:
        Location.append((Start_Lat[i], Start_Lng[i]))
WI_data['Location'] = Location

# Add a new row: 'Duration(hour)'
Start_Time = [row for row in WI_data.loc[:, 'Start_Time']]
End_Time = [row for row in WI_data.loc[:, 'End_Time']]
Duration = []
for i in range(len(End_Time)):
    if Start_Time[i] == None or End_Time[i] == None:
        Duration.append(None)
    else:
        start = datetime.strptime(Start_Time[i][:19],"%Y-%m-%d %H:%M:%S")
        end = datetime.strptime(End_Time[i][:19],"%Y-%m-%d %H:%M:%S")
        Duration.append(int((end - start).seconds/60))
WI_data['Duration(minutes)'] = Duration

# Add a new row: 'Distance'
Distance = []
for dist in WI_data['Distance(mi)']:
    Distance.append(int(dist*1000))
WI_data['Distance'] = Distance

In [5]:
# List all the attributes that possibly have an influence in the decision
X_names=['Amenity',
'Crossing',
'Give_Way',
'No_Exit',
'Junction',
'Railway',
'Roundabout',
'Station',
'Stop',
'Traffic_Calming',
'Traffic_Signal',
'Turning_Loop',
'Severity',
'Distance',
'Duration(minutes)',
'Location',
'Time']


for name in X_names:
    new_col = copy.deepcopy(WI_data[name])
    count = 0
    data = pd.concat([data, new_col], axis = 1)
data.set_index(['ID'], inplace = True)
data

,Amenity,Crossing,Give_Way,No_Exit,Junction,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Severity,Distance,Duration(minutes),Location,Time
ID,,,,,,,,,,,,,,,,,
A-11933,False,False,False,False,False,False,False,False,False,False,False,False,2,37,360,"(43.767345, -91.152598)",18
A-12224,False,False,False,False,True,False,False,False,False,False,False,False,2,753,360,"(44.91181, -91.31724)",8
A-12294,False,False,False,False,False,False,False,False,False,False,False,False,2,497,360,"(44.815139, -91.4545)",18
A-12663,False,False,False,False,True,False,False,False,False,False,False,False,4,8991,360,"(44.9213, -92.1213)",6
A-12675,False,False,False,False,True,False,False,False,False,False,False,False,2,1012,360,"(44.57523, -91.20188)",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A-2844607,False,False,False,False,False,False,False,False,False,False,True,False,4,70,27,"(42.49961, -89.03361)",0
A-2844608,False,False,False,False,False,False,False,False,False,False,True,False,4,70,27,"(42.500616, -89.03365699999998)",0
A-2845028,False,False,False,False,False,False,False,False,False,False,False,False,3,0,28,"(42.50133, -89.03328)",6


In [6]:
# Feature Selection
def get_max(lis):
    max_v = 0
    for i in lis:
        if math.isnan(i) == False and i > max_v:
            max_v = i
    return max_v

X_raw = data.iloc[:,:12]
y_raw = data.loc[:,'Severity']
selector0 = SelectKBest(chi2, k=3) 
selector0.fit(X_raw, y_raw)
max0 = get_max(selector0.scores_)

y_raw = data.loc[:,'Distance']
selector1 = SelectKBest(chi2, k=3) 
selector1.fit(X_raw, y_raw)
max1 = get_max(selector1.scores_)

y_raw = data.loc[:,'Duration(minutes)']
selector2 = SelectKBest(chi2, k=3) 
selector2.fit(X_raw, y_raw)
max2 = get_max(selector2.scores_)

y_raw = data.loc[:,'Time']
selector3 = SelectKBest(chi2, k=3) 
selector3.fit(X_raw, y_raw)
max3 = get_max(selector3.scores_)

threshold = 0.2
Bad = []
print("The average weighted score of each attributes for predicting:")
for i in range(len(selector0.scores_)):
    avg_score = (selector0.scores_[i]/max0 + selector1.scores_[i]/max1 + selector2.scores_[i]/max2 + selector3.scores_[i]/max3)/4
    print(X_names[i]+':', avg_score)
    if avg_score < threshold or math.isnan(avg_score) == True:
        Bad.append(X_names[i])
print("\nIneffective Features:",Bad)
for bad in Bad:
    data = data.drop(bad, axis = 1)

The average weighted score of each attributes for predicting:
Amenity: 0.44736741524948054
Crossing: 0.677849008728475
Give_Way: 0.13772716416716524
No_Exit: 0.275311384680919
Junction: 0.5616788127516553
Railway: 0.2604456926320655
Roundabout: nan
Station: 0.28911303075396727
Stop: 0.3408760617232638
Traffic_Calming: 0.11702836153761967
Traffic_Signal: 0.25748375633868686
Turning_Loop: nan

Ineffective Features: ['Give_Way', 'Roundabout', 'Traffic_Calming', 'Turning_Loop']


In [7]:
data

,Amenity,Crossing,No_Exit,Junction,Railway,Station,Stop,Traffic_Signal,Severity,Distance,Duration(minutes),Location,Time
ID,,,,,,,,,,,,,
A-11933,False,False,False,False,False,False,False,False,2,37,360,"(43.767345, -91.152598)",18
A-12224,False,False,False,True,False,False,False,False,2,753,360,"(44.91181, -91.31724)",8
A-12294,False,False,False,False,False,False,False,False,2,497,360,"(44.815139, -91.4545)",18
A-12663,False,False,False,True,False,False,False,False,4,8991,360,"(44.9213, -92.1213)",6
A-12675,False,False,False,True,False,False,False,False,2,1012,360,"(44.57523, -91.20188)",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
A-2844607,False,False,False,False,False,False,False,True,4,70,27,"(42.49961, -89.03361)",0
A-2844608,False,False,False,False,False,False,False,True,4,70,27,"(42.500616, -89.03365699999998)",0
A-2845028,False,False,False,False,False,False,False,False,3,0,28,"(42.50133, -89.03328)",6
